In [ ]:
#| default_exp experimental

# experimental

> Experimental `dialoghelper` capabilities.

In [ ]:
#| export

from importlib import resources
import uuid
from fasthtml.common import Div, Script
from time import sleep
import base64
import json
import time
from claudette import ToolResult
from dialoghelper.core import *
from dataclasses import dataclass
from httpx import get as xget, post as xpost

In [ ]:
#| export

_js_loaded = False

def _load_screenshot_js():
    "Load screenshot capability if not already present."
    global _js_loaded
    if _js_loaded:
        print("screenshot.js already loaded")
        return
    js_content = (resources.files('dialoghelper') / 'screenshot.js').read_text()
    add_html(Div(Script(js_content), hx_swap_oob='beforeend:#js-script'))
    print("Added screenshot.js")
    _js_loaded = True

In [ ]:
#| export

_screen_share_active = False

def start_screen_share(timeout=30):
  "Start persistent screen sharing session, waiting for confirmation."
  global _screen_share_active
  _load_screenshot_js()
  status_id = str(uuid.uuid4())
  start_code = f'startPersistentScreenShare("{status_id}");'
  trigger_script = create_iife(start_code)
  add_html(Div(Script(trigger_script), hx_swap_oob=f'beforeend:#js-script'))
  print("Waiting for screen share permission...")
  start_time = time.time()
  max_attempts = int(timeout * 2)  # Poll every 0.5s
  for attempt in range(max_attempts):
      result = xpost('http://localhost:5001/pop_data_', data={'data_id': status_id})
      if result.status_code == 200 and result.text.strip():
          try:
              data = result.json()
              js_status = data.get('js_status')
              if js_status == 'connected':
                  _screen_share_active = True
                  print("✅ Screen share permission granted!")
                  return "Screen share started successfully."
              elif js_status == 'error':
                  error_msg = data.get('error', 'Unknown error')
                  print(f"❌ Screen share failed: {error_msg}")
                  return f"Screen share failed: {error_msg}"
              elif js_status == 'connecting':
                  print("⏳ Still waiting for permission...")
          except json.JSONDecodeError: pass
      time.sleep(0.5)
  print("⏰ Screen share permission timed out")
  return f"Screen share timed out after {timeout} seconds."

def stop_screen_share():
  "Stop persistent screen sharing session."
  global _screen_share_active
  stop_code = 'stopPersistentScreenShare();'
  trigger_script = create_iife(stop_code)
  add_html(Div(Script(trigger_script), hx_swap_oob=f'beforeend:#js-script'))
  _screen_share_active = False
  return "Screen share stopped."

In [ ]:
#| export

def create_iife(code: str) -> str:
    "Wrap javascript code string in an IIFE"
    return f'''
(async () => {{
{code}
}})();
'''

In [ ]:
#| export

def capture_screen():
  "Capture screenshot, automatically starting screen share if needed."
  global _screen_share_active
  _load_screenshot_js()
  if not _screen_share_active:
      print("🔄 No active screen share, starting one...")
      result = start_screen_share()
      if not _screen_share_active: return f"Failed to start screen share: {result}"              
  data_id = str(uuid.uuid4())
  screenshot_code = f'captureScreenFromStream("{data_id}");'
  print("📸 Capturing from persistent stream...")
  trigger_script = create_iife(screenshot_code)
  add_html(Div(Script(trigger_script), hx_swap_oob=f'beforeend:#js-script'))
  max_attempts = 20
  for attempt in range(max_attempts):
      result = xpost('http://localhost:5001/pop_data_', data={'data_id': data_id})
      if result.status_code == 200:
          try:
              if not result.text.strip():
                  time.sleep(0.5)
                  continue
              data = result.json()
              if 'error' in data: return f"Screenshot failed: {data['error']}"
              if 'img_data' in data and 'img_type' in data: return ToolResult(data=data['img_data'], result_type=data['img_type'])
              return f"Screenshot data incomplete: {data}"
          except json.JSONDecodeError as e: print(f"JSON decode error: {e}, Raw text: '{result.text}'")
          except Exception as e: print(f"Error processing screenshot data: {e}")
      time.sleep(0.5)
  return "Screenshot capture timed out"